In [1]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.classic import SplitMNIST

from avalanche.evaluation.metrics import forgetting_metrics, \
accuracy_metrics, loss_metrics, timing_metrics, cpu_usage_metrics, \
confusion_matrix_metrics, disk_usage_metrics, bwt_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger, WandBLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training import Naive,EWC
from torchvision.transforms import Compose, ToTensor, Normalize, RandomCrop, Resize
import numpy as np
import random
import wandb
import timm
import torch
import avalanche

/home/grbagwe/miniconda3/envs/fedconlearn/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
seed = 2023
np.random.seed(seed)
torch.manual_seed(seed)
random.seed(seed)
torch.cuda.manual_seed(seed)


In [3]:

train_transform = Compose([
    Resize(224),
    ToTensor(),
])

test_transform = Compose([
    Resize(224),
    ToTensor(),
])


benchmark = avalanche.benchmarks.SplitCIFAR100(n_experiences=10, return_task_id=False, seed=seed, 
                       train_transform=train_transform,eval_transform = test_transform
                      )

Files already downloaded and verified
Files already downloaded and verified


In [4]:
benchmark.original_classes_in_exp

[{5, 9, 23, 37, 39, 48, 58, 62, 86, 96},
 {7, 18, 19, 28, 33, 41, 67, 69, 78, 99},
 {17, 29, 36, 44, 47, 66, 74, 80, 83, 88},
 {6, 20, 21, 27, 43, 51, 81, 82, 85, 90},
 {1, 4, 14, 63, 65, 73, 84, 91, 93, 95},
 {2, 10, 12, 32, 53, 54, 59, 75, 89, 92},
 {31, 34, 35, 40, 42, 45, 55, 64, 97, 98},
 {0, 8, 24, 38, 49, 52, 68, 71, 79, 87},
 {15, 16, 22, 25, 30, 50, 56, 57, 61, 94},
 {3, 11, 13, 26, 46, 60, 70, 72, 76, 77}]

In [8]:
benchmark.get_reproducibility_data()

{'class_ids_from_zero_from_first_exp': False,
 'class_ids_from_zero_in_each_exp': False,
 'class_mapping': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99],
 'classes_order': [39,
  48,
  62,
  58,
  37,
  9,
  5,
  86,
  96,
  23,
  41,
  69,
  18,
  7,
  67,
  78,
  99,
  28,
  19,
  33,
  17,
  36,
  29,
  80,
  88,
  44,
  66,
  47,
  83,
  74,
  51,
  81,
  90,
  20,
  82,
  27,
  85,
  6,
  43,
  21,
  95,
  14,
  84,
  1,
  73,
  91,
  4,
  93,
  63

In [5]:
# model = timm.models.vit_tiny_patch16_224(pretrained=True, num_classes=benchmark.n_classes)
model =timm.models.resnet18(pretrained= True, num_classes = benchmark.n_classes)
model = torch.nn.DataParallel(model)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /home/grbagwe/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


In [6]:
model

DataParallel(
  (module): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (drop_block): Identity()
        (act1): ReLU(inplace=True)
        (aa): Identity()
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): ReLU(inplace=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(

In [9]:
# loggers 
loggers = []

# log to Tensorboard
loggers.append(TensorboardLogger())

# log to text file
loggers.append(TextLogger(open('log.txt', 'a')))

# print to stdout
loggers.append(InteractiveLogger())

# W&B logger - comment this if you don't have a W&B account
loggers.append(WandBLogger(project_name="avalanche", run_name="ewc-SplitCifar100-Resnet18"))

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
#     timing_metrics(epoch=True, epoch_running=True),
#     cpu_usage_metrics(experience=True),
    forgetting_metrics(experience=True, stream=True),
    bwt_metrics(experience=True, stream=True),
    confusion_matrix_metrics(num_classes=benchmark.n_classes, save_image=True,
                             stream=True),
#     disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=loggers
)


In [10]:
# CREATE THE STRATEGY INSTANCE (EWC)
cl_strategy = EWC(
    model,
    optimizer=SGD(model.parameters(), lr=0.0001),
    criterion=CrossEntropyLoss(),
    ewc_lambda=0.7,
    train_mb_size=50, train_epochs=10,
    eval_mb_size=50,device='cuda',
    evaluator=eval_plugin)

In [11]:
# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in benchmark.train_stream:
    print("Start training on experience ", experience.current_experience)

    cl_strategy.train(experience)
    print("End training on experience", experience.current_experience)
    print("Computing accuracy on the test set")
    results.append(cl_strategy.eval(benchmark.test_stream[:]))
        
        
# for experience in benchmark.train_stream:
#     # train returns a dictionary which contains all the metric values
#     res = cl_strategy.train(experience)
#     print('Training completed')

#     print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
#     results.append(cl_strategy.eval(benchmark.test_stream))
    


Starting experiment...
Start training on experience  0
-- >> Start of training phase << --
100%|██████████████████████████████████████████████████████████████████| 100/100 [00:24<00:00,  4.08it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 4.6991
	Loss_MB/train_phase/train_stream/Task000 = 4.4503
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0066
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.0000
100%|██████████████████████████████████████████████████████████████████| 100/100 [00:13<00:00,  7.61it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 4.2566
	Loss_MB/train_phase/train_stream/Task000 = 4.1327
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0438
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.0400
100%|██████████████████████████████████████████████████████████████████| 100/100 [00:13<00:00,  7.45it/s]
Epoch 2 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 3.8510
	Loss_MB/train_phase/train_stream/Task000 = 3.5931
	Top1

100%|██████████████████████████████████████████████████████████████████| 100/100 [00:15<00:00,  6.65it/s]
Epoch 2 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 4.2436
	Loss_MB/train_phase/train_stream/Task000 = 4.0752
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0026
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.0000
100%|██████████████████████████████████████████████████████████████████| 100/100 [00:14<00:00,  6.78it/s]
Epoch 3 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 3.8226
	Loss_MB/train_phase/train_stream/Task000 = 3.5596
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0296
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.0400
100%|██████████████████████████████████████████████████████████████████| 100/100 [00:14<00:00,  6.69it/s]
Epoch 4 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 3.4703
	Loss_MB/train_phase/train_stream/Task000 = 3.4179
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.1102
	Top1_Acc_MB/train_phase/train_stream/

100%|██████████████████████████████████████████████████████████████████| 100/100 [00:16<00:00,  6.18it/s]
Epoch 4 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 3.5837
	Loss_MB/train_phase/train_stream/Task000 = 3.4611
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0682
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.1000
100%|██████████████████████████████████████████████████████████████████| 100/100 [00:16<00:00,  6.24it/s]
Epoch 5 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 3.2478
	Loss_MB/train_phase/train_stream/Task000 = 3.1982
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.2184
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.2200
100%|██████████████████████████████████████████████████████████████████| 100/100 [00:16<00:00,  6.07it/s]
Epoch 6 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 2.9696
	Loss_MB/train_phase/train_stream/Task000 = 2.8343
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.3376
	Top1_Acc_MB/train_phase/train_stream/

	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.1306
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.2200
100%|██████████████████████████████████████████████████████████████████| 100/100 [00:17<00:00,  5.71it/s]
Epoch 6 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 3.0668
	Loss_MB/train_phase/train_stream/Task000 = 2.8580
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.2770
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.4600
100%|██████████████████████████████████████████████████████████████████| 100/100 [00:17<00:00,  5.81it/s]
Epoch 7 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 2.8209
	Loss_MB/train_phase/train_stream/Task000 = 2.6503
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.3478
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.3200
100%|██████████████████████████████████████████████████████████████████| 100/100 [00:17<00:00,  5.66it/s]
Epoch 8 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 2.6065
	Loss_MB/train_phase/train_stream/

100%|██████████████████████████████████████████████████████████████████| 100/100 [00:18<00:00,  5.33it/s]
Epoch 7 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 2.9145
	Loss_MB/train_phase/train_stream/Task000 = 2.7286
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.2910
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.3200
100%|██████████████████████████████████████████████████████████████████| 100/100 [00:18<00:00,  5.36it/s]
Epoch 8 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 2.7334
	Loss_MB/train_phase/train_stream/Task000 = 2.6246
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.3196
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.4000
100%|██████████████████████████████████████████████████████████████████| 100/100 [00:18<00:00,  5.36it/s]
Epoch 9 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 2.5735
	Loss_MB/train_phase/train_stream/Task000 = 2.5158
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.3506
	Top1_Acc_MB/train_phase/train_stream/

100%|██████████████████████████████████████████████████████████████████| 100/100 [00:43<00:00,  2.27it/s]
Epoch 8 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 2.6491
	Loss_MB/train_phase/train_stream/Task000 = 2.5981
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.4300
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.4600
100%|██████████████████████████████████████████████████████████████████| 100/100 [00:44<00:00,  2.27it/s]
Epoch 9 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 2.4556
	Loss_MB/train_phase/train_stream/Task000 = 2.2854
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.4576
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.5200
-- >> End of training phase << --
End training on experience 5
Computing accuracy on the test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|████████████████████████████████████████████████████████████████████| 20/20 [00:07<00:00,  2.76it/s]
> Eval on experience 0 (

	Loss_MB/train_phase/train_stream/Task000 = 2.6506
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.3262
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.4800
100%|██████████████████████████████████████████████████████████████████| 100/100 [00:39<00:00,  2.56it/s]
Epoch 9 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 2.6715
	Loss_MB/train_phase/train_stream/Task000 = 2.7386
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.3564
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.2800
-- >> End of training phase << --
End training on experience 6
Computing accuracy on the test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|████████████████████████████████████████████████████████████████████| 20/20 [00:07<00:00,  2.55it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/Exp000 = -0.5670
	ExperienceForgetting/eval_phase/test_stream/Task000/Exp000 = 0.5670
	Loss_Exp/e

	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.5356
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.6800
100%|██████████████████████████████████████████████████████████████████| 100/100 [00:48<00:00,  2.07it/s]
Epoch 9 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 2.2884
	Loss_MB/train_phase/train_stream/Task000 = 2.2261
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.5752
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.6200
-- >> End of training phase << --
End training on experience 7
Computing accuracy on the test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|████████████████████████████████████████████████████████████████████| 20/20 [00:06<00:00,  2.98it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/Exp000 = -0.5670
	ExperienceForgetting/eval_phase/test_stream/Task000/Exp000 = 0.5670
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 5.3744
	Top1

	Loss_MB/train_phase/train_stream/Task000 = 2.6685
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.4242
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.4000
100%|██████████████████████████████████████████████████████████████████| 100/100 [00:51<00:00,  1.95it/s]
Epoch 9 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 2.5644
	Loss_MB/train_phase/train_stream/Task000 = 2.5345
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.4774
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.4400
-- >> End of training phase << --
End training on experience 8
Computing accuracy on the test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|████████████████████████████████████████████████████████████████████| 20/20 [00:06<00:00,  2.90it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/Exp000 = -0.5670
	ExperienceForgetting/eval_phase/test_stream/Task000/Exp000 = 0.5670
	Loss_Exp/e

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:45<00:00,  2.18it/s]
Epoch 6 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 3.3809
	Loss_MB/train_phase/train_stream/Task000 = 3.1509
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.2436
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.3000
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:58<00:00,  1.71it/s]
Epoch 7 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 3.0700
	Loss_MB/train_phase/train_stream/Task000 = 2.9123
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.3610
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.4600
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████

In [12]:
import wandb
wandb.finish()

ExperienceBWT/eval_phase/test_stream/Task000/Exp000,▁▁▁▁▁▁▁▁▁
ExperienceBWT/eval_phase/test_stream/Task000/Exp001,█▁▁▁▁▁▁▁
ExperienceBWT/eval_phase/test_stream/Task000/Exp002,█▁▁▁▁▁▁
ExperienceBWT/eval_phase/test_stream/Task000/Exp003,█▁▁▁▁▁
ExperienceBWT/eval_phase/test_stream/Task000/Exp004,█▂▁▁▁
ExperienceBWT/eval_phase/test_stream/Task000/Exp005,█▁▁▁
ExperienceBWT/eval_phase/test_stream/Task000/Exp006,█▁▁
ExperienceBWT/eval_phase/test_stream/Task000/Exp007,█▁
ExperienceBWT/eval_phase/test_stream/Task000/Exp008,▁
ExperienceForgetting/eval_phase/test_stream/Task000/Exp000,▁▁▁▁▁▁▁▁▁
ExperienceForgetting/eval_phase/test_stream/Task000/Exp001,▁███████
